In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.svm import SVC
from joblib import dump,load
import os
from sklearn.metrics import accuracy_score, classification_report
def fetch_data(ticker):
    stock = yf.Ticker(ticker)
    data = stock.history(period="max")  # Fetch 5 years of data
    data.reset_index(inplace=True)
    return data


In [2]:
stock_list=pd.read_csv('nasdaq-listed.csv')

In [3]:

def preprocess_data(data):
    # Feature engineering: Create relevant columns
    data['open-close'] = data['Open'] - data['Close']
    data['high-low'] = data['High'] - data['Low']
    data['price-change'] = data['Close'].pct_change()
    data['is_quarter_end'] = np.where((data['Date'].dt.month % 3 == 0) & (data['Date'].dt.day > 23), 1, 0)
    data['SMA_10'] = data['Close'].rolling(window=10).mean()
    data['SMA_50'] = data['Close'].rolling(window=50).mean()
    data['SMA_200'] = data['Close'].rolling(window=200).mean()
    data['EMA_10'] = data['Close'].ewm(span=10, adjust=False).mean()
    data['EMA_50'] = data['Close'].ewm(span=50, adjust=False).mean()
    data['EMA_200'] = data['Close'].ewm(span=200, adjust=False).mean()
    # Drop unnecessary columns
    data = data.drop(['Dividends', 'Stock Splits'], axis=1, errors='ignore')
    data.dropna(inplace=True)  # Handle missing values
    
    return data




In [4]:
accuracy_list=pd.read_csv("Accuracy_Data_NASDAQ.csv")

print(accuracy_list)

      STOCK   Accuracy Symbol
0       NaN  55.911330   AACG
1       NaN  54.248366   AADI
2       NaN  53.197674   AADR
3       NaN  49.354839    AAL
4       NaN  67.598017   AAME
...     ...        ...    ...
1140    NaN  52.322738   EDIT
1141    NaN  52.459016   EDOC
1142    NaN  56.164384   EDRY
1143    NaN  62.427746   EDSA
1144    NaN  54.347826   EDTK

[1145 rows x 3 columns]


In [6]:
for index, row in stock_list.iloc[1569:].iterrows():
    stock=row['Stocks']
    data=fetch_data(stock)
    if (not data.empty) & (len(data)>11):
        data=preprocess_data(data)
        model_dir=f"./models/{stock}/"
        if not os.path.exists(model_dir):
                    os.mkdir(model_dir)
        accuracy_list.to_csv("Accuracy_Data_NASDAQ.csv",index=False)
        # Define features (X) and target (y)
        features = data[['open-close', 'high-low', 'Volume', 'is_quarter_end','SMA_10','SMA_50',"SMA_200",'EMA_10','EMA_50',"EMA_200"]]
        target = np.where(data['Close'].shift(-1) > data['Close'], 1, 0)  # 1 foSr price increase, 0 otherwise
        if features.shape[0] > 0 and target.shape[0] > 0:
   
            X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)
            
            # Standardize features
            scaler = StandardScaler()
            X_train = scaler.fit_transform(X_train)
            X_test = scaler.transform(X_test)
            # Initialize and train the model
            model = SVC(kernel='poly', probability=True, random_state=42)  
            model.fit(X_train, y_train)
            
            # Save the scaler and model for reuse
            
            dump(scaler, f"{model_dir}{stock}_scaler.joblib")
            dump(model, f"{model_dir}{stock}_predictor.joblib")
            y_pred = model.predict(X_test)
            
            # Evaluate performance
            print(stock," ",index)
            print("Accuracy:", accuracy_score(y_test, y_pred))
            print("Classification Report:\n", classification_report(y_test, y_pred))
            stock_accuracy=pd.DataFrame({"Symbol":[row['Stocks']],"Accuracy":[accuracy_score(y_test,y_pred)*100]})
            accuracy_list=pd.concat([accuracy_list,stock_accuracy])
            print("Accuracy List Length :",len(accuracy_list))
        

FINW   1569
Accuracy: 0.5897435897435898
Classification Report:
               precision    recall  f1-score   support

           0       0.62      0.86      0.72        73
           1       0.38      0.14      0.20        44

    accuracy                           0.59       117
   macro avg       0.50      0.50      0.46       117
weighted avg       0.53      0.59      0.53       117

Accuracy List Length : 1362
FINX   1570
Accuracy: 0.5105820105820106
Classification Report:
               precision    recall  f1-score   support

           0       0.29      0.01      0.02       182
           1       0.51      0.97      0.67       196

    accuracy                           0.51       378
   macro avg       0.40      0.49      0.35       378
weighted avg       0.40      0.51      0.36       378

Accuracy List Length : 1363
FIP   1571
Accuracy: 0.4642857142857143
Classification Report:
               precision    recall  f1-score   support

           0       0.50      0.24      0.

FLDDW: Period 'max' is invalid, must be one of ['1d', '5d']


FLEX   1588
Accuracy: 0.5129053606882858
Classification Report:
               precision    recall  f1-score   support

           0       0.51      0.98      0.67       766
           1       0.62      0.03      0.06       745

    accuracy                           0.51      1511
   macro avg       0.57      0.51      0.37      1511
weighted avg       0.57      0.51      0.37      1511

Accuracy List Length : 1380
FLGC   1589
Accuracy: 0.5555555555555556
Classification Report:
               precision    recall  f1-score   support

           0       0.56      0.98      0.71        82
           1       0.00      0.00      0.00        62

    accuracy                           0.56       144
   macro avg       0.28      0.49      0.36       144
weighted avg       0.32      0.56      0.41       144

Accuracy List Length : 1381
FLGT   1590
Accuracy: 0.5
Classification Report:
               precision    recall  f1-score   support

           0       0.51      0.85      0.63       192
 

FMSTW: Period 'max' is invalid, must be one of ['1d', '5d']


FNGR   1611
Accuracy: 0.6077844311377245
Classification Report:
               precision    recall  f1-score   support

           0       0.61      0.98      0.75       200
           1       0.67      0.04      0.08       134

    accuracy                           0.61       334
   macro avg       0.64      0.51      0.42       334
weighted avg       0.63      0.61      0.48       334

Accuracy List Length : 1400
FNK   1612
Accuracy: 0.46615384615384614
Classification Report:
               precision    recall  f1-score   support

           0       0.35      0.04      0.07       335
           1       0.47      0.92      0.62       315

    accuracy                           0.47       650
   macro avg       0.41      0.48      0.35       650
weighted avg       0.41      0.47      0.34       650

Accuracy List Length : 1401
FNKO   1613
Accuracy: 0.4735202492211838
Classification Report:
               precision    recall  f1-score   support

           0       0.49      0.26      0

...

ILLRW: Period 'max' is invalid, must be one of ['1d', '5d']


ILMN   2169
Accuracy: 0.5243697478991597
Classification Report:
               precision    recall  f1-score   support

           0       0.54      0.04      0.07       569
           1       0.52      0.97      0.68       621

    accuracy                           0.52      1190
   macro avg       0.53      0.50      0.38      1190
weighted avg       0.53      0.52      0.39      1190

Accuracy List Length : 1870
ILPT   2170
Accuracy: 0.5016077170418006
Classification Report:
               precision    recall  f1-score   support

           0       0.49      0.96      0.65       148
           1       0.70      0.09      0.15       163

    accuracy                           0.50       311
   macro avg       0.59      0.52      0.40       311
weighted avg       0.60      0.50      0.39       311

Accuracy List Length : 1871
IMAB   2171
Accuracy: 0.5714285714285714
Classification Report:
               precision    recall  f1-score   support

           0       0.58      0.93      0

IMTXW: Period 'max' is invalid, must be one of ['1d', '5d']


IMUX   2191
Accuracy: 0.528
Classification Report:
               precision    recall  f1-score   support

           0       0.53      0.94      0.68       267
           1       0.45      0.06      0.10       233

    accuracy                           0.53       500
   macro avg       0.49      0.50      0.39       500
weighted avg       0.49      0.53      0.41       500

Accuracy List Length : 1891
IMVT   2192
Accuracy: 0.4435146443514644
Classification Report:
               precision    recall  f1-score   support

           0       0.44      0.63      0.52       115
           1       0.44      0.27      0.34       124

    accuracy                           0.44       239
   macro avg       0.44      0.45      0.43       239
weighted avg       0.44      0.44      0.43       239

Accuracy List Length : 1892
IMXI   2193
Accuracy: 0.5511363636363636
Classification Report:
               precision    recall  f1-score   support

           0       0.55      0.93      0.69       191

ValueError: The number of classes has to be greater than one; got 1 class

In [ ]:
accuracy_list.to_csv("Accuracy_Data_NASDAQ.csv",index=False)